In [22]:
# from preprocess import TextReader
from train import CNNText

from app import _get_available_dev, _buildVocabulary

# import pandas as pd
# from pyemd import emd
import numpy as np
# from gensim.models import KeyedVectors
# from sklearn.model_selection import train_test_split
# from model import Model

In [2]:
_get_available_dev('CPU')

['/device:CPU:0']

In [3]:
_get_available_dev('GPU')

[]

## 1. Process the data and convert words into vectors

In [4]:
!ls ./data/

ranks  rt-polarity.neg	rt-polarity.pos  train	valid


In [5]:
_buildVocabulary(base_path='./data/',
                 clean=True,
                 max_vocab=15000,
                 shuffle=True,
                 test_size=0.1)

Found datafiles with the following class labels {'./data/rt-polarity.pos': 1, './data/rt-polarity.neg': 0}


Processing: ./data/rt-polarity.pos:   0%|          | 0/2 [00:00<?, ?it/s]

Created token ranks ./data/ranks of size 15000


Processing: ./data/rt-polarity.neg: 100%|██████████| 2/2 [00:05<00:00,  2.46s/it]

Created training data of shape (10662, 51)
Created training label of shape (10662,)
Saved the train and test frames in ./data/


In [6]:
!ls ./data/

ranks  rt-polarity.neg	rt-polarity.pos  train	valid


In [10]:
X_train = np.load('./data/train/X_train.npy')
X_valid = np.load('./data/valid/X_valid.npy')

In [13]:
y_train = np.load('./data/train/y_train.npy')
y_valid = np.load('./data/valid/y_valid.npy')

In [14]:
X_train.shape

(9595, 51)

In [15]:
X_valid.shape

(1067, 51)

In [16]:
y_train.shape

(9595,)

In [17]:
y_valid.shape

(1067,)

## Testing the batch Iterators

In [21]:
!mkdir TFLOGS

In [23]:
cnnText = CNNText(
    train_path='./train/',
    valid_path='./valid/',
    epochs=5,
    batch_size=50
)

In [24]:
cnnText.train(tolerance=0.01, logdir='./TFLOGS/',)

step 10/955 (epoch 1/5), loss = 8.800632 (859.5 examples/sec; 0.058 sec/batch), lr: 0.010000
step 20/955 (epoch 1/5), loss = 5.031693 (948.2 examples/sec; 0.053 sec/batch), lr: 0.010000
step 30/955 (epoch 1/5), loss = 2.993293 (982.7 examples/sec; 0.051 sec/batch), lr: 0.010000
step 40/955 (epoch 1/5), loss = 1.927647 (920.8 examples/sec; 0.054 sec/batch), lr: 0.010000
step 50/955 (epoch 1/5), loss = 1.323100 (971.1 examples/sec; 0.051 sec/batch), lr: 0.010000
step 60/955 (epoch 1/5), loss = 1.041734 (967.9 examples/sec; 0.052 sec/batch), lr: 0.010000
step 70/955 (epoch 1/5), loss = 0.864123 (988.2 examples/sec; 0.051 sec/batch), lr: 0.010000
step 80/955 (epoch 1/5), loss = 0.772862 (980.7 examples/sec; 0.051 sec/batch), lr: 0.010000
step 90/955 (epoch 1/5), loss = 0.761071 (928.9 examples/sec; 0.054 sec/batch), lr: 0.010000
step 100/955 (epoch 1/5), loss = 0.722306 (941.6 examples/sec; 0.053 sec/batch), lr: 0.010000
step 110/955 (epoch 1/5), loss = 0.700774 (957.5 examples/sec; 0.052 

step 840/955 (epoch 5/5), loss = 0.673110 (867.1 examples/sec; 0.058 sec/batch), lr: 0.010000
step 850/955 (epoch 5/5), loss = 0.688016 (879.5 examples/sec; 0.057 sec/batch), lr: 0.010000
step 860/955 (epoch 5/5), loss = 0.692029 (881.7 examples/sec; 0.057 sec/batch), lr: 0.010000
step 870/955 (epoch 5/5), loss = 0.664616 (883.3 examples/sec; 0.057 sec/batch), lr: 0.010000
step 880/955 (epoch 5/5), loss = 0.666635 (941.8 examples/sec; 0.053 sec/batch), lr: 0.010000
step 890/955 (epoch 5/5), loss = 0.693246 (989.8 examples/sec; 0.051 sec/batch), lr: 0.010000
step 900/955 (epoch 5/5), loss = 0.710077 (874.0 examples/sec; 0.057 sec/batch), lr: 0.010000
step 910/955 (epoch 5/5), loss = 0.692272 (894.9 examples/sec; 0.056 sec/batch), lr: 0.010000
step 920/955 (epoch 5/5), loss = 0.677874 (880.2 examples/sec; 0.057 sec/batch), lr: 0.010000
step 930/955 (epoch 5/5), loss = 0.673888 (924.3 examples/sec; 0.054 sec/batch), lr: 0.010000
step 940/955 (epoch 5/5), loss = 0.690818 (938.7 examples/se